# Import Modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
from sklearn.inspection import permutation_importance
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.calibration import calibration_curve

# Loading Dataset

In [ ]:
df = pd.read_csv('SRSavg.csv')

df.info()

# Data Preprocessing

In [ ]:
feature_columns = ['SRS_RAW_TOTAL', 'SRS_AWARENESS', 'SRS_COGNITION', 'SRS_COMMUNICATION', 'SRS_MOTIVATION', 'SRS_MANNERISMS']
target_column = 'HAS ADHD'

X = df[feature_columns]
y = df[target_column]

df.head()

# Split the Data Into Testing and Training

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the Model

In [ ]:
svm_model = SVC(probability=True)

# Best parameters: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
param_grid = { 'C': [1, 10, 100],
              'gamma': [1, 0.1, 0.01],
              'kernel': ['rbf', 'linear']
}

# Using RandomizedSearchCV to find the best parameters.
random_search = RandomizedSearchCV(svm_model, param_distributions=param_grid, n_iter=5, cv=5, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# Searching for the best model
print(f"Best Parameters: {random_search.best_params_}")
best_model = random_search.best_estimator_

# Predicting the Data

In [9]:
y_pred = best_model.predict(X_test)

# Conduct Accuracy and Report

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}")

report = classification_report(y_test, y_pred)
print(report)

# Evaluating the Model

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure()
sns.heatmap(cm, annot=True, fmt="d", cmap="Reds", xticklabels=["Not ADHD", "ADHD"], yticklabels=["Not ADHD", "ADHD"])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title("Confusion Matrix")
plt.show()

# Correlation matrix
corr = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', xticklabels=corr.columns, yticklabels=corr.columns)
plt.title("Correlation Matrix of SRS Scores")
plt.show()


# Feature importance
perm_importance = permutation_importance(best_model, X, y, n_repeats=30, random_state=42, n_jobs=-1)
sorted_idx = perm_importance.importances_mean.argsort()
indices = np.argsort(perm_importance)[::-1]

plt.figure(figsize=(10, 6))
plt.barh(range(X.shape[1]), perm_importance.importances_mean[sorted_idx], align='center')
plt.yticks(range(X.shape[1]), X.columns[sorted_idx])
plt.xlabel("Permutation Importance")
plt.title("Feature Importance of SVC Model")
plt.show()


# ROC curve
y_score = best_model.predict_proba(X_test)[:, 1] 
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr) 

plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:0.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', label='Line of No Skill')  
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


# Calibration curve
y_prob = best_model.predict_proba(X_test)[:, 1]

prob_true, prob_pred = calibration_curve(y_test, y_prob, n_bins=10)
plt.plot(prob_pred, prob_true, marker='o', label='SVC')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')  
plt.title("Calibration Curve")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("Fraction of Positives")
plt.legend()
plt.show()

# Pair plot
sns.pairplot(df, hue='HAS ADHD', markers='o')
plt.suptitle('Pair Plot of SRS Features and ADHD Predictions', y=1.02)
plt.show()


# Visualising the Model Prediction

In [ ]:
# Gradio function to predict ADHD based on SRS scores
def classify(SRS_AWARENESS, SRS_COGNITION, SRS_COMMUNICATION, SRS_MOTIVATION, SRS_MANNERISMS):
    srs_scores = [SRS_AWARENESS, SRS_COGNITION, SRS_COMMUNICATION, SRS_MOTIVATION, SRS_MANNERISMS]
    total_srs_score = sum(srs_scores)
    
    input_data = pd.DataFrame(
        [[total_srs_score] + srs_scores],
        columns=['SRS_RAW_TOTAL', 'SRS_AWARENESS', 'SRS_COGNITION', 'SRS_COMMUNICATION', 'SRS_MOTIVATION', 'SRS_MANNERISMS'])

    prediction = best_model.predict(input_data)[0]
    probability = best_model.predict_proba(input_data)[0]
    
    result = "No ADHD" if prediction == 0 else "ADHD"
    prob_result = f"ADHD Probability: {probability[1]:.2f}%"

    return result, prob_result

# Gradio interface to interact with the model.
interface = gr.Interface(
    fn=classify,
    inputs = [
        gr.Number(label="SRS Awareness Score"),
        gr.Number(label="SRS Cognition Score"),
        gr.Number(label="SRS Communication Score"),
        gr.Number(label="SRS Motivation Score"),
        gr.Number(label="SRS Mannerisms Score")
    ],
    outputs = [
        gr.Textbox(label="Classification Result"),
        gr.Textbox(label="Probability Score") ],
    title="ADHD Classification",
    description="Enter the SRS scores to get an ADHD classification and probability."
)

interface.launch(share=True)